## Chapter 21: ASP.NET Core with a Modern JavaScript Frontend

So far, we’ve built applications where the server renders HTML using Razor views. However, modern web development often separates the frontend and backend: the backend serves as a Web API, and the frontend is a **Single Page Application (SPA)** built with frameworks like React, Angular, or Vue. This architecture offers a richer user experience, better separation of concerns, and the ability to reuse the same API for mobile apps or third‑party clients. In this chapter, you’ll learn how to integrate ASP.NET Core with a modern JavaScript frontend. We’ll cover serving static files, enabling CORS, using the ASP.NET Core SPA templates, and structuring your application as a backend API with a separate frontend project. By the end, you’ll be able to build full‑stack applications that combine the power of ASP.NET Core with the flexibility of modern JavaScript frameworks.

### 21.1 SPA vs. Traditional MVC

In a traditional MVC application:
- The server handles routing, authentication, and data access.
- Views are rendered on the server and sent as HTML to the browser.
- Each user action (e.g., clicking a link) typically triggers a full page reload.

In a **Single Page Application** (SPA):
- The server provides a Web API that returns data (usually JSON), not HTML.
- The frontend is a client‑side application that runs in the browser. It handles rendering, routing, and user interactions.
- After the initial page load, the SPA communicates with the API via AJAX/fetch requests, updating only parts of the page without full reloads.

**Benefits of SPA:**
- **Rich interactivity:** The UI feels more responsive and app‑like.
- **Separation of concerns:** Frontend and backend can be developed, deployed, and scaled independently.
- **Reusable API:** The same API can power a web SPA, a mobile app, and third‑party integrations.
- **Better developer experience:** Frontend developers can use modern tools (hot reload, component libraries) without being tied to server‑side rendering.

**Challenges:**
- Initial load time may be higher (JavaScript bundle size).
- SEO can be trickier (though frameworks address this with server‑side rendering or prerendering).
- Requires more sophisticated state management and client‑side routing.

### 21.2 ASP.NET Core as a Pure Backend API

The most common pattern is to build your ASP.NET Core application as a **Web API** (as covered in Chapter 10) and host your SPA separately, either on the same server or on a different domain. However, you can also serve the SPA static files directly from ASP.NET Core, which simplifies deployment (everything in one project).

#### Option 1: Separate Frontend and Backend Projects

This is the most flexible approach. Your backend is a Web API project, and your frontend is a separate project (e.g., created with Create React App, Vue CLI, or Angular CLI). During development, you run both servers (e.g., ASP.NET Core on `localhost:5000` and React on `localhost:3000`) and use a proxy or CORS to communicate.

**Pros:**
- Clean separation.
- Frontend can use its own build tools and development server with hot reload.
- Easier to deploy to different hosting environments (e.g., frontend on a CDN, backend on Azure).

**Cons:**
- Requires CORS configuration.
- Deployment involves two separate artifacts.

#### Option 2: Frontend Served by ASP.NET Core

You can build your SPA and place the compiled static files in ASP.NET Core’s `wwwroot` folder, then serve them using `UseStaticFiles`. The API endpoints are under `/api` routes, and you configure a fallback route so that any non‑API request returns the `index.html` (so client‑side routing works).

**Pros:**
- Single deployment unit.
- No CORS issues (same origin).
- Simple for small teams or projects.

**Cons:**
- Frontend build process must be integrated (e.g., as part of the .NET build).
- Less flexibility in hosting.

In this chapter, we’ll focus on **Option 2** (serving the SPA from ASP.NET Core) because it’s straightforward to set up and demonstrates the integration points. We’ll also discuss CORS for Option 1.

### 21.3 Creating the Project and Setting Up the SPA

Let’s create a new ASP.NET Core Web API project and add a React frontend.

#### Step 1: Create the API Project

```bash
dotnet new webapi -n MyApp
cd MyApp
```

#### Step 2: Add a React Frontend

You can use the `dotnet new` template that includes a React SPA, but we’ll do it manually to understand the pieces.

First, create a `ClientApp` folder at the root of the project. Inside, initialize a React app (using Create React App, Vite, or your preferred tool). For example, using Vite:

```bash
npm create vite@latest ClientApp -- --template react
cd ClientApp
npm install
```

This creates a React application in the `ClientApp` folder.

#### Step 3: Configure ASP.NET Core to Serve the SPA

We need to:
- Serve static files from the `ClientApp/dist` folder (after building).
- Redirect all non‑API requests to `index.html` so that client‑side routing works.

First, add the SPA services and middleware. The `Microsoft.AspNetCore.SpaServices.Extensions` package provides helper methods. It’s already referenced in some templates, but if not, add it:

```bash
dotnet add package Microsoft.AspNetCore.SpaServices.Extensions
```

In `Program.cs`:

```csharp
var builder = WebApplication.CreateBuilder(args);

builder.Services.AddControllers();
builder.Services.AddSpaStaticFiles(configuration =>
{
    configuration.RootPath = "ClientApp/dist";
});

var app = builder.Build();

app.UseHttpsRedirection();
app.UseStaticFiles(); // Serve static files from wwwroot
app.UseSpaStaticFiles(); // Serve static files from the SPA folder

app.UseRouting();

app.UseEndpoints(endpoints =>
{
    endpoints.MapControllers();
});

app.UseSpa(spa =>
{
    spa.Options.SourcePath = "ClientApp";

    if (app.Environment.IsDevelopment())
    {
        // Use the React development server
        spa.UseReactDevelopmentServer(npmScript: "dev"); // or "start" for CRA
    }
});

app.Run();
```

**Explanation:**
- `AddSpaStaticFiles` configures the folder where the production SPA files will be located (`ClientApp/dist`).
- `UseStaticFiles` serves files from `wwwroot` (if any).
- `UseSpaStaticFiles` serves the SPA static files.
- `UseSpa` sets up the SPA middleware. In development, it can launch the frontend dev server (via `npm run dev`). In production, it simply serves the static files from the build output.

#### Step 4: Build the React App for Production

When deploying, you need to build the React app and ensure the output goes to `ClientApp/dist`. You can automate this in your CI/CD pipeline.

#### Step 5: API Endpoints

Add your API controllers under `/api/...`. For example, a simple `WeatherForecastController` (as in the default template) will be available at `/api/weatherforecast`.

#### Step 6: Client‑Side Routing

In your React app, you likely use React Router. Ensure that for any route not handled by the API, the server returns `index.html`. The `UseSpa` middleware does exactly that: it intercepts requests that don’t match static files or API endpoints and returns the `index.html` from the SPA folder.

### 21.4 CORS (Cross‑Origin Resource Sharing)

If you choose to host your frontend separately (e.g., React on `localhost:3000` and API on `localhost:5000`), you must enable **CORS** in your ASP.NET Core API so that the browser allows requests from the frontend origin.

#### Adding CORS

In `Program.cs`:

```csharp
builder.Services.AddCors(options =>
{
    options.AddPolicy("AllowReactApp",
        policy =>
        {
            policy.WithOrigins("http://localhost:3000") // React dev server
                  .AllowAnyHeader()
                  .AllowAnyMethod()
                  .AllowCredentials(); // if using cookies/auth
        });
});

// ... after app building
app.UseCors("AllowReactApp");
```

Order matters: `UseCors` must be called after `UseRouting` and before `UseAuthorization` and `UseEndpoints`.

#### CORS with Multiple Origins

For production, you may need to allow your deployed frontend domain(s). You can read allowed origins from configuration.

```csharp
var allowedOrigins = builder.Configuration.GetSection("AllowedOrigins").Get<string[]>();
options.AddPolicy("AllowFrontend", policy =>
    policy.WithOrigins(allowedOrigins)
          .AllowAnyHeader()
          .AllowAnyMethod());
```

#### Preflight Requests

For requests with custom headers or methods other than simple ones (GET, POST with certain content types), the browser sends an OPTIONS preflight request. CORS middleware automatically handles this if you include `AllowAnyHeader` and `AllowAnyMethod`.

### 21.5 Development Experience: Using a Proxy

When running frontend and backend separately in development, you often want to avoid CORS and just proxy API requests through the frontend dev server. For example, with Vite, you can configure a proxy in `vite.config.js`:

```js
export default {
  server: {
    proxy: {
      '/api': 'https://localhost:5001' // your ASP.NET Core API URL
    }
  }
}
```

Then in your React code, you can make requests to `/api/weatherforecast` and they will be proxied to the backend, avoiding CORS entirely.

### 21.6 Authentication with SPA and API

Authenticating a SPA with a backend API requires a token‑based approach (usually JWT) because cookies can be tricky with cross‑origin requests (though possible with careful configuration). We covered JWT authentication in Chapter 13.

For a SPA, you typically:
- Obtain a JWT token from a login endpoint (`/api/auth/login`).
- Store the token (in memory, or in a secure, httpOnly cookie if using cookie authentication).
- Include the token in the `Authorization` header for subsequent API requests.

**Example login flow:**

```javascript
const login = async (email, password) => {
  const response = await fetch('/api/auth/login', {
    method: 'POST',
    headers: { 'Content-Type': 'application/json' },
    body: JSON.stringify({ email, password })
  });
  const data = await response.json();
  localStorage.setItem('token', data.token); // Not the most secure, but common
};

const fetchData = async () => {
  const token = localStorage.getItem('token');
  const response = await fetch('/api/protected', {
    headers: { 'Authorization': `Bearer ${token}` }
  });
  // ...
};
```

For better security, consider using httpOnly cookies with the `SameSite` attribute, but that requires the API and frontend to be on the same domain or careful CORS configuration.

### 21.7 Integrating Frontend Build into .NET Build

If you choose to serve the SPA from ASP.NET Core, you may want to trigger the frontend build during the .NET build process. You can achieve this by editing the `.csproj` file:

```xml
<Target Name="BuildFrontend" BeforeTargets="Build">
  <Exec Command="npm install" WorkingDirectory="ClientApp" />
  <Exec Command="npm run build" WorkingDirectory="ClientApp" />
</Target>
```

This ensures that when you build the .NET project, the frontend is also built, and the output is placed in `ClientApp/dist` (or wherever your build outputs). Then `UseSpaStaticFiles` will serve those files.

### 21.8 Full Example: React + ASP.NET Core API

Let’s create a minimal React component that fetches and displays data from our API.

**React Component (`App.jsx`):**

```jsx
import React, { useState, useEffect } from 'react';

function App() {
  const [forecasts, setForecasts] = useState([]);

  useEffect(() => {
    fetch('/api/weatherforecast')
      .then(response => response.json())
      .then(data => setForecasts(data));
  }, []);

  return (
    <div>
      <h1>Weather Forecast</h1>
      <ul>
        {forecasts.map((f, index) => (
          <li key={index}>{f.date}: {f.temperatureC}°C, {f.summary}</li>
        ))}
      </ul>
    </div>
  );
}

export default App;
```

**ASP.NET Core API Controller (`WeatherForecastController`):**

```csharp
[ApiController]
[Route("api/[controller]")]
public class WeatherForecastController : ControllerBase
{
    private static readonly string[] Summaries = new[]
    {
        "Freezing", "Bracing", "Chilly", "Cool", "Mild", "Warm", "Balmy", "Hot", "Sweltering", "Scorching"
    };

    [HttpGet]
    public IEnumerable<WeatherForecast> Get()
    {
        var rng = new Random();
        return Enumerable.Range(1, 5).Select(index => new WeatherForecast
        {
            Date = DateTime.Now.AddDays(index),
            TemperatureC = rng.Next(-20, 55),
            Summary = Summaries[rng.Next(Summaries.Length)]
        })
        .ToArray();
    }
}
```

With the SPA middleware configured as above, running `dotnet run` in development will start both the API and the React dev server (if configured). Navigating to the root URL will serve the React app, and the API requests from React will be handled by the same server (or proxied).

### 21.9 The "Web API + SPA" Architecture Pattern

This pattern is widely adopted. Key takeaways:

- **API First**: Design your API to be consumed by any client (web, mobile, third‑party). Use RESTful principles or GraphQL.
- **Clear Separation**: Keep your frontend project independent. It can use its own build tools, state management, and routing.
- **Communication**: Use HTTP with JSON. For real‑time, consider SignalR.
- **Security**: Use token‑based authentication (JWT) and ensure CORS is properly configured.
- **Deployment**: You can deploy the frontend and backend together (as we did) or separately (e.g., frontend on static hosting, backend on cloud).

### 21.10 Performance Considerations

- **Bundle Size**: Optimize your JavaScript bundles (code splitting, tree shaking).
- **API Payload Size**: Compress responses, use pagination, and only send necessary data.
- **Caching**: Use HTTP caching headers, and consider caching API responses (see Chapter 20).
- **Lazy Loading**: Load parts of the application only when needed.

### Summary

In this chapter, you learned how to integrate ASP.NET Core with modern JavaScript frontends:

- The difference between traditional MVC and SPA architectures.
- Two approaches: serving the SPA from ASP.NET Core or keeping them separate.
- How to configure the SPA middleware to serve React/Vue/Angular apps.
- How to enable CORS when the frontend and backend are on different origins.
- Authentication considerations and the use of JWT.
- Building and deploying the full‑stack application.

With this knowledge, you can build powerful web applications that combine the robustness of ASP.NET Core with the rich interactivity of modern JavaScript frameworks.

**Exercise:**

1. Create a new ASP.NET Core Web API project and add a React frontend using Vite or Create React App in a `ClientApp` folder.
2. Configure the SPA middleware to serve the React app in development (using the dev server) and production (using built files).
3. Add a simple API endpoint that returns a list of items, and display them in a React component.
4. If you have time, set up CORS for a separate frontend project (e.g., React on port 3000) and test cross‑origin requests.

In the next chapter, **"gRPC vs. SignalR vs. Web APIs,"** we’ll compare different communication protocols and when to use each. You’ll learn about gRPC for high‑performance internal communication, SignalR for real‑time features, and traditional Web APIs for RESTful services.